In [1]:
import random
import math
import scipy.special as spc
from itertools import cycle

---
## XOR cipher with random bytes generator

In [2]:
def random_bytes_generator():
    """ Generates random bytes """
    while True:
        yield random.getrandbits(8)

def xor_encipher(plaintext: str, key: str) -> str:
    """
    Applies XOR cipher by xor'ing binary representation of each char with key.
    Works for any unicode char.
    """
    random.seed(key)
    return ''.join([
        chr(ord(char) ^ k)
        for char, k in zip(plaintext, random_bytes_generator())
    ])

def xor_decipher(ciphertext: str, key: str) -> str:
    """
    Decrypts XOR cipher applying it secound time.
    Works for any unicode char.
    """
    return xor_encipher(ciphertext, key)

### Example

In [3]:
xor_encipher('ala ma kota', 'klucz')

"1¢\x83'ß«\x17È°Ý\x13"

In [4]:
xor_decipher("1¢\x83'ß«\x17È°Ý\x13", 'klucz')

'ala ma kota'

### Tests on XOR cipher

In [5]:
plain_1 = 'abcdefg'
plain_2 = 'abcrtyu'

cipher_1 = xor_encipher(plain_1, 'klucz')
cipher_2 = xor_encipher(plain_2, 'klucz')

plain_xor = [ord(c1)^ord(c2) for c1, c2 in zip(cipher_1, cipher_2)]
cipher_xor = [ord(c1)^ord(c2) for c1, c2 in zip(plain_1, plain_2)]

In [6]:
plain_xor

[0, 0, 0, 22, 17, 31, 18]

In [7]:
cipher_xor

[0, 0, 0, 22, 17, 31, 18]

In [8]:
plain_xor == cipher_xor

True

---
## 1. Frequency (Monobit) Test

In [9]:
def frequency(series: str) -> float:
    """
    Note that this description is taken from the NIST documentation [1]
  
    The focus of this test is the proportion of zeros and ones for the entire sequence. The purpose of this test is
    to determine whether the number of ones and zeros in a sequence are approximately the same as would be expected
    for a truly random sequence. This test assesses the closeness of the fraction of ones to 1/2, that is the number
    of ones and zeros ina  sequence should be about the same. All subsequent tests depend on this test.
  
    :param text: a binary string <=> string that contains only '0' and '1'
    :return: the p-value from the test
    """
    n = len(series)
    
    # (1)
    S = series.count('1') - series.count('0')
    
    # (2)
    sobs = S / math.sqrt(n)
    
    # (3)
    p_value = spc.erfc(math.fabs(sobs) / math.sqrt(2))
    
    return p_value

def validate_frequency(series: str) -> bool:
    """
    As described in NIST documentation [1]:
    
    If the computed P-value is < 0.01, then conclude that the sequence is non-random.
    Otherwise, conclude that the sequence is random. 
    """
    return frequency(series) >= 0.01

### Examples

In [10]:
samples = ['1011010101', '0'*100, '1'*100, '1110'*25, '10'*50, '1'*50+'0'*50]
samples += [''.join(random.choices('01', k=100)) for _ in range(5)]

for s in samples:
    print(f"'{s}'\nWynik: {frequency(s):1.12f}  {validate_frequency(s)}\n")

'1011010101'
Wynik: 0.527089256866  True

'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'
Wynik: 0.000000000000  False

'1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111'
Wynik: 0.000000000000  False

'1110111011101110111011101110111011101110111011101110111011101110111011101110111011101110111011101110'
Wynik: 0.000000573303  False

'1010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010'
Wynik: 1.000000000000  True

'1111111111111111111111111111111111111111111111111100000000000000000000000000000000000000000000000000'
Wynik: 1.000000000000  True

'1110010001100100110111101100010111111011101000001111100000111000011011100110101010001111111010101101'
Wynik: 0.230139340443  True

'0000100110110111001100110001011010010000001100111001111100010000110101000000101011101100110000000000'
Wynik: 0.071860638226  True

'110010001100110011110010100100

---
## 2. Frequency Test within a Block

In [11]:
def block_frequency(series: str, M: int) -> float:
    """
    Note that this description is taken from the NIST documentation [1]
  
    The focus of the test is the proportion of ones within M-bit blocks. The purpose of this test is to determine
    whether the frequency of ones in an M-bit block is approximately M/2, as would be expected under an
    assumption of randomness. For block size M=1, this test degenerates to test 1, the Frequency (Monobit)
    test. 
  
    :param series: a binary string <=> string that contains only '0' and '1'
    :param M: size of block
    :return: the p-value from the test
    """
    n = len(series)
    
    # (1)
    blocks = [series[i:i+M] for i in range(0, n-1, M)]
    N = len(blocks)
    
    # (2)
    blocks_pi = [block.count('1') / M for block in blocks]
    
    # (3)
    chi_squared = 4 * M * sum((pi - 0.5)**2 for pi in blocks_pi)
    
    # (4)
    p_value = spc.gammaincc(N / 2, chi_squared / 2)
    
    return p_value

def validate_block_frequency(series: str, M: int) -> bool:
    """
    As described in NIST documentation [1]:
    
    If the computed P-value is < 0.01, then conclude that the sequence is non-random.
    Otherwise, conclude that the sequence is random. 
    """
    return block_frequency(series, M) >= 0.01

### Examples

In [12]:
samples = [('0110011010', 3), ('0'*100, 20), ('1'*100, 20), ('1110'*25, 20), ('10'*50, 20), ('1'*50+'0'*50, 20)]
samples += [(''.join(random.choices('01', k=100)), 20) for _ in range(5)]

for s, m in samples:
    print(f"M={m}\t'{s}'\nWynik: {block_frequency(s, m):1.12f}  {validate_block_frequency(s, m)}\n")

M=3	'0110011010'
Wynik: 0.801251956901  True

M=20	'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'
Wynik: 0.000000000000  False

M=20	'1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111'
Wynik: 0.000000000000  False

M=20	'1110111011101110111011101110111011101110111011101110111011101110111011101110111011101110111011101110'
Wynik: 0.000139333791  False

M=20	'1010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010'
Wynik: 1.000000000000  True

M=20	'1111111111111111111111111111111111111111111111111100000000000000000000000000000000000000000000000000'
Wynik: 0.000000000000  False

M=20	'1000110101101001000101010100011000100101011010101101001110001100111101011000100001101001000011111010'
Wynik: 0.962565773247  True

M=20	'1011110000100111111100001001100101111111011011111101000011011000110101110011111101000101010110101000'
Wynik: 0.36903586957

---
## 3. Runs Test

In [13]:
def runs(text: str) -> float:
    """
    Note that this description is taken from the NIST documentation [1]
  
    The focus of this test is the total number of runs in the sequence, where a run is an uninterrupted sequence
    of identical bits. A run of length k consists of exactly k identical bits and is bounded before and after with
    a bit of the opposite value. The purpose of the runs test is to determine whether the number of runs of ones and zeros of various lengths is as expected for a random sequence. In particular, this test determines
    whether the oscillation between such zeros and ones is too fast or too slow.
  
    :param series: a binary string <=> string that contains only '0' and '1'
    :return: the p-value from the test
    """
    n = len(text)
    
    # (1)
    pi = text.count('1') / len(text)
    
    # (2)
    if abs(pi - 0.5) >= 2 / math.sqrt(n):
        return 0.0
    
    # (3)
    vobs = sum(text[k] != text[k+1] for k in range(len(text)-1)) + 1
    
    # (4)
    p_value = spc.erfc(abs(vobs - 2 * n * pi * (1 - pi)) / (2 * math.sqrt(2 * n) * pi * (1 - pi)))
    
    return p_value

def validate_runs(series: str) -> bool:
    """
    As described in NIST documentation [1]:
    
    If the computed P-value is < 0.01, then conclude that the sequence is non-random.
    Otherwise, conclude that the sequence is random. 
    """
    return runs(series) >= 0.01

In [14]:
samples = ['1001101011', '0'*100, '1'*100, '1110'*25, '10'*50, '1'*50+'0'*50]
samples += [''.join(random.choices('01', k=100)) for _ in range(5)]

for s in samples:
    print(f"'{s}'\nWynik: {runs(s):1.12f}  {validate_runs(s)}\n")

'1001101011'
Wynik: 0.147232255364  True

'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'
Wynik: 0.000000000000  False

'1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111'
Wynik: 0.000000000000  False

'1110111011101110111011101110111011101110111011101110111011101110111011101110111011101110111011101110'
Wynik: 0.000000000000  False

'1010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010'
Wynik: 0.000000000000  False

'1111111111111111111111111111111111111111111111111100000000000000000000000000000000000000000000000000'
Wynik: 0.000000000000  False

'1011010110001010101111011001101100100101100101000110101010101011000010111010010010110000001000000101'
Wynik: 0.007336755458  False

'0000001101100011110000111011001000000001000111000011110010001110010100011100111000000001110000000010'
Wynik: 0.010106728213  True

'001000100010000001111001000

---
## 4. Test for the Longest Run of Ones in a Block

In [15]:
def longest_run_of_ones(series: str) -> float:
    """
    Note that this description is taken from the NIST documentation [1]
  
    The focus of the test is the longest run of ones within M-bit blocks. The purpose of this test is to
    determine whether the length of the longest run of ones within the tested sequence is consistent with the
    length of the longest run of ones that would be expected in a random sequence. Note that an irregularity in
    the expected length of the longest run of ones implies that there is also an irregularity in the expected
    length of the longest run of zeroes. Therefore, only a test for ones is necessary.
    
    Assumes values: M = 8, K = 3 and N = 16
  
    :param series: a binary string <=> string that contains only '0' and '1'
    :return: the p-value from the test
    """
    M = 8
    K = 3
    N = 16
    pi = [0.2148, 0.3672, 0.2305, 0.1875]
    n = len(series)
    
    # (1)
    blocks = [series[i:i+M] for i in range(0, n-1, M)]
    N = len(blocks)
    
    # (2)
    lengths = [
        max(len(s) for s in block.split('0'))
        for block in blocks
    ]
    v = [
        sum(l <= 1 for l in lengths),
        sum(l == 2 for l in lengths),
        sum(l == 3 for l in lengths),
        sum(l >= 4 for l in lengths)
    ]
    
    # (3)
    chi_squared = sum((v[i] - N * pi[i])**2 / (N * pi[i]) for i in range(K+1))
    
    # (4)
    p_value = spc.gammaincc(K / 2, chi_squared / 2)
    
    return p_value

def validate_longest_run_of_ones(series: str) -> bool:
    """
    As described in NIST documentation [1]:
    
    If the computed P-value is < 0.01, then conclude that the sequence is non-random.
    Otherwise, conclude that the sequence is random. 
    """
    return longest_run_of_ones(series) >= 0.01

In [16]:
samples = ['11001100000101010110110001001100111000000000001001001101010100010001001111010110100000001101011111001100111001101101100010110010']
samples += ['0'*100, '1'*100, '1110'*25, '10'*50, '1'*50+'0'*50]
samples += [''.join(random.choices('01', k=100)) for _ in range(5)]

for s in samples:
    print(f"'{s}'\nWynik: {runs(s):1.12f}  {validate_runs(s)}\n")

'11001100000101010110110001001100111000000000001001001101010100010001001111010110100000001101011111001100111001101101100010110010'
Wynik: 0.620728953394  True

'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'
Wynik: 0.000000000000  False

'1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111'
Wynik: 0.000000000000  False

'1110111011101110111011101110111011101110111011101110111011101110111011101110111011101110111011101110'
Wynik: 0.000000000000  False

'1010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010101010'
Wynik: 0.000000000000  False

'1111111111111111111111111111111111111111111111111100000000000000000000000000000000000000000000000000'
Wynik: 0.000000000000  False

'0010011001011100011110100101100110110010011000100010100111110110001111110000001000010011110111010101'
Wynik: 1.000000000000  True

'111100100000000100010001010000010101000010

## References
[1] http://csrc.nist.gov/publications/nistpubs/800-22-rev1a/SP800-22rev1a.pdf